# Machine Learning Prototype


We'll start off with a simple model like logistic regression. This is interpretable and explainable, so it should be a solid baseline to get an understanding before we move onto more complicated models.

Because there is a major class imbalance of mostly samples that have not failed, we will implement stratified k-fold cross validation. This ensures that each fold maintains the same proportion of observations for each target class as the complete dataset.

Additionally, because we saw earlier that some metrics have very high means and standard deviations, we will normalize these metrics. This will allow for the model to predict more accurately by not incorrectly weighing features more simply because they have potentially extreme values. 

Finally, as mentioned earlier, there is a strong class imbalance. In order to correct this, we will oversample. This makes it so that there are an equal number of failures as non failures in the training data. This is done by random sampling from the failure data until there an equal number of data points of both classes.

Note, that since we are applying K-fold cross validation, it is important to apply the oversampling after we split the data with cross validation. If we do it before, i.e. oversample, then do k-fold cross validation, there will potentially be data leakage. This is because oversampling takes data that already exists and duplicates it. So there will be duplicate rows, thus potentially having one instance be in the training data, and one in the test data, which will lead to overfitting. 



In [34]:
nonNumericColumns = ["device", "date", "lastRecordedDate"]

engineered_df = df.drop(nonNumericColumns, axis = 1)

X,y = engineered_df.drop(columns = ["failure"]), engineered_df.failure

print(X.shape)
X.head()

(124488, 10)


,metric1,log_m2,log_m3,log_m4,log_m8,log_m9,device_category,day,month,day_week
0,215630672,4.025352,0.000000,3.970292,0.0,2.079442,S1F0,1,1,3
1,61370680,0.000000,1.386294,0.000000,0.0,0.000000,S1F0,1,1,3
2,173295968,0.000000,0.000000,0.000000,0.0,0.000000,S1F0,1,1,3
3,79694024,0.000000,0.000000,0.000000,0.0,0.000000,S1F0,1,1,3
4,135970480,0.000000,0.000000,0.000000,0.0,1.386294,S1F0,1,1,3


In [35]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: failure, dtype: int64

Below we're going to create a class specifically for the columns of our feature (X) matrix. We need to normalize metric1 of the data because it is so large, but not other metrics as they are ordinal categorical features. 

You may be wondering, why don't we just normalize that column right now, before we split the data into train and test? We don't do that because that will cause data leakage, i.e. the test data normalization for metric1 will have included our train data, which will skew our results and potentially lead to incorrect accuracy metrics.

In [36]:
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

numeric = ['metric1']
categorical = ['log_m2', 'log_m3', 'log_m4', 'log_m8', 'log_m9', 'day', 'month', 'day_week']

In [37]:
def preprocess(X,y,OHE,low_card,scaler,i,sm):
    samples = X.copy()
    labels = y.copy()
    
    samples.reset_index(inplace = True, drop = True)
    labels.reset_index(inplace = True, drop = True)
    
    if i == 1:
        #Encoding categorical variables for train set
        coding_hot = OHE.fit_transform(samples[low_card].to_numpy())
        aux = pd.DataFrame(coding_hot, columns = OHE.get_feature_names_out(low_card))
        samples = pd.concat([samples, aux], axis=1).drop(low_card, axis=1)
        # Scaling as features
        aux = scaler.fit_transform(samples)
        samples = pd.DataFrame(aux,index= samples.index, columns= samples.columns)
        xf, yf = sm.fit_resample(samples, labels) 
        
    else:
        #Encoding categorical variables for test set
        coding_hot = OHE.transform(samples[low_card].to_numpy())
        aux = pd.DataFrame(coding_hot, columns = OHE.get_feature_names_out(low_card))
        samples = pd.concat([samples, aux], axis=1).drop(low_card, axis=1)
        # Scaling as features
        aux2 = scaler.transform(samples)
        xf = pd.DataFrame(aux2,index=samples.index, columns=samples.columns)
        yf = y
    
    return xf, yf


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

X_train.reset_index(inplace = True, drop = True)
y_train.reset_index(inplace = True, drop = True)

X_test.reset_index(inplace = True, drop = True)
y_test.reset_index(inplace = True, drop = True)


In [39]:
X_train.head()

,metric1,log_m2,log_m3,log_m4,log_m8,log_m9,device_category,day,month,day_week
0,210331464,0.0,0.000000,0.0,0.0,0.000000,W1F1,10,1,5
1,241893544,0.0,0.000000,0.0,0.0,0.000000,W1F0,19,7,6
2,159871712,0.0,5.398163,0.0,0.0,3.218876,S1F0,16,1,4
3,243540872,0.0,0.000000,0.0,0.0,0.000000,W1F1,31,5,6
4,58172424,0.0,0.000000,0.0,0.0,0.000000,Z1F0,4,1,6


In [40]:
X_train.describe()

,metric1,log_m2,log_m3,log_m4,log_m8,log_m9,day,month,day_week
count,8.714100e+04,87141.000000,87141.000000,87141.000000,87141.000000,87141.000000,87141.000000,87141.000000,87141.000000
mean,1.225075e+08,0.319076,0.177434,0.164197,0.031064,0.396357,14.905831,4.027702,3.009938
std,7.050994e+07,1.445548,0.822763,0.651295,0.299585,1.005679,8.750221,2.569862,1.999602
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,6.138375e+07,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,2.000000,1.000000
50%,1.227866e+08,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,3.000000,3.000000
75%,1.836940e+08,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,6.000000,5.000000
max,2.441405e+08,11.081666,7.898782,7.418781,6.725034,9.836386,31.000000,11.000000,6.000000


In [41]:
X_train.nunique()

metric1            86783
log_m2               487
log_m3                45
log_m4               104
log_m8                26
log_m9                64
device_category        7
day                   31
month                 11
day_week               7
dtype: int64

In [42]:
print(X_train.columns)
print(X_train.shape)
print(X_train.dtypes)

Index(['metric1', 'log_m2', 'log_m3', 'log_m4', 'log_m8', 'log_m9',
       'device_category', 'day', 'month', 'day_week'],
      dtype='object')
(87141, 10)
metric1              int64
log_m2             float64
log_m3             float64
log_m4             float64
log_m8             float64
log_m9             float64
device_category     object
day                  int32
month                int32
day_week             int32
dtype: object


In [43]:
from sklearn.compose import ColumnTransformer
columns_to_encode = ['device_category']
columns_to_scale  = ['metric1']

OHE =  OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)

scaler = StandardScaler()

#Oversampler to help with class imbalance
sm = SMOTE(random_state=0)

#First drop any non numeric categories
pipeline=ColumnTransformer([
    ('num',scaler,columns_to_scale),
    ('cat',OHE,columns_to_encode),
    
])

new_X_train= pipeline.fit_transform(X_train)
new_X_test = pipeline.transform(X_test)

In [44]:
ohe_columns = pipeline.get_feature_names_out()
X_train_encoded = pd.DataFrame(new_X_train, index = X_train.index, columns=ohe_columns)
X_test_encoded =  pd.DataFrame(new_X_test, index = X_test.index, columns=ohe_columns)

X_train.drop(columns=["device_category", "metric1"], inplace=True)
X_test.drop(columns=["device_category", "metric1"], inplace=True)

X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

X_train, y_train = sm.fit_resample(X_train, y_train)
X_train.head()

,log_m2,log_m3,log_m4,log_m8,log_m9,day,month,day_week,num__metric1,cat__device_category_S1F0,cat__device_category_S1F1,cat__device_category_W1F0,cat__device_category_W1F1,cat__device_category_Z1F0,cat__device_category_Z1F1,cat__device_category_Z1F2
0,0.0,0.000000,0.0,0.0,0.000000,10,1,5,1.245562,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.000000,19,7,6,1.693190,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,5.398163,0.0,0.0,3.218876,16,1,4,0.529917,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.000000,31,5,6,1.716553,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.000000,4,1,6,-0.912431,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [45]:
print(y_train.value_counts())

failure
0    87072
1    87072
Name: count, dtype: int64


In [46]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(174144, 16) (174144,)
(37347, 16) (37347,)


## Model Selection and Evaluation
Given the imbalanced nature of our predictive maintenance dataset, where most data represents devices that have not failed, it's crucial to choose evaluation metrics that are robust to class imbalance and prioritize the correct identification of the minority class (failed devices) while minimizing false positives.

We'll look at several different models to train on this dataset.

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

import warnings


from sklearn.metrics import roc_auc_score

### Gradient Boosting Classifier
* Gradient Boosting Classifier is an ensemble learning method that combines multiple weak learners (typically decision trees) to create a strong predictive model.
* It is effective at capturing complex interactions between features and target variables, making it suitable for predictive maintenance datasets with non-linear relationships and interactions.
* Gradient Boosting can handle missing data and outliers effectively, which are common challenges in predictive maintenance datasets, and it automatically selects relevant features, reducing the need for manual feature engineering.
#### Parameters:
* n_estimators: The number of boosting stages (trees) to be used. Increasing this parameter generally improves performance at the cost of increased computational complexity.
* learning_rate: The shrinkage parameter that controls the contribution of each tree to the final prediction. Smaller values require more trees to achieve comparable performance but may improve generalization.
* max_depth: The maximum depth of the individual trees. This parameter controls the complexity of the trees and helps prevent overfitting.
* min_samples_split and min_samples_leaf: These parameters control the minimum number of samples required to split an internal node and the minimum number of samples required to be at a leaf node, respectively. They help prevent overfitting by controlling tree complexity.
* subsample: The fraction of samples to be used for fitting the individual base learners. It introduces randomness into the training process and can improve generalization.
* max_features: The number of features to consider when looking for the best split. By tuning this parameter, we can control the randomness in feature selection for each tree.

In [48]:
# Define your parameter grid for Gradient Boosting
param_grid_gb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Initialize the Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier()

# GridSearchCV with 5-fold cross-validation
grid_search_gb = GridSearchCV(estimator=gb_classifier, param_grid=param_grid_gb, cv=5, n_jobs=-1, verbose=2)

# Fit the GridSearchCV on your training data
grid_search_gb.fit(X_train, y_train)

# Get the best parameters
best_params_gb = grid_search_gb.best_params_

# Predict labels for the test data
y_pred_gb = grid_search_gb.predict(X_test)


Fitting 5 folds for each of 27 candidates, totalling 135 fits


## Logistic Regression
LR with 5 fold grid search cross-validation. Parameters:
* C: Regularization parameter. Similar to LinearSVC, it controls the trade-off between fitting the training data and preventing overfitting.
* penalty: The norm of the penalty used in regularization. Options include 'l1' (Lasso) and 'l2' (Ridge). These penalties can help reduce overfitting by shrinking the coefficients towards zero.
* class_weight: Similar to LinearSVC, this parameter allows us to handle class imbalance by adjusting the penalty associated with misclassifying each class.
* solver: The optimization algorithm to use. Different solvers have different characteristics and may perform differently depending on the dataset.

In [53]:
# Define your parameter grid for Logistic Regression
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Initialize the Logistic Regression Classifier
lr_classifier = LogisticRegression()

# GridSearchCV with 5-fold cross-validation
grid_search_lr = GridSearchCV(estimator=lr_classifier, param_grid=param_grid_lr, cv=5, n_jobs=-1, verbose=2)

# Fit the GridSearchCV on your training data
grid_search_lr.fit(X_train, y_train)

# Get the best parameters
best_params_lr = grid_search_lr.best_params_

# Predict labels for the test data
y_pred_lr = grid_search_lr.predict(X_test)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


### Multi-Layer Neural Network with ReLU Activation

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score
import numpy as np

# Define your training data (X_train) and labels (y_train)
# Define your testing data (X_test) and labels (y_test)

# Convert your data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define your neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Assuming 2 classes for classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize your neural network
input_size = X_train.shape[1]  # Number of features
model = NeuralNetwork(input_size)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define batch size and number of epochs
batch_size = 128
num_epochs = 10

# Create DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    model.eval()
    outputs = model(X_test_tensor)
    _, y_pred_nn = torch.max(outputs, 1)

Epoch [1/10], Loss: 0.0419
Epoch [2/10], Loss: 0.0121
Epoch [3/10], Loss: 0.0258
Epoch [4/10], Loss: 0.0069
Epoch [5/10], Loss: 0.0022
Epoch [6/10], Loss: 0.0031
Epoch [7/10], Loss: 0.0065
Epoch [8/10], Loss: 0.0020
Epoch [9/10], Loss: 0.0008
Epoch [10/10], Loss: 0.0042


In [56]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve

# Define a dictionary to store evaluation metrics for each model
evaluation_metrics = {}

# Define a list of metrics to calculate
metrics = ['Precision', 'Recall', 'F1 Score', 'Accuracy', 'Specificity', 'AUC-ROC', 'AUC-PR']

# Calculate metrics for each model
for model_name, y_pred in zip(['Gradient Boosting', 'Logistic Regression', 'Neural Network'], 
                              [y_pred_gb, y_pred_lr, y_pred_nn]):
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    roc_auc = roc_auc_score(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    auc_roc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    auc_pr = auc(recall, precision)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    evaluation_metrics[model_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Specificity': specificity,
        'AUC-ROC': auc_roc,
        'AUC-PR': auc_pr
    }

# Print metrics for each model
for model_name, metrics_dict in evaluation_metrics.items():
    print(f"Metrics for {model_name}:")
    for metric_name, metric_value in metrics_dict.items():
        print(metric_name, ":",round(metric_value, 4))
    print()


Metrics for Random Forest:
Precision : 0.0833
Recall : 0.027
F1 Score : 0.0408
Accuracy : 0.9987
Specificity : 0.9997
AUC-ROC : 0.5134
AUC-PR : 0.0557

Metrics for Gradient Boosting:
Precision : 0.1667
Recall : 0.0811
F1 Score : 0.1091
Accuracy : 0.9987
Specificity : 0.9996
AUC-ROC : 0.5403
AUC-PR : 0.1243

Metrics for SVM:
Precision : 0.0
Recall : 0.0
F1 Score : 0.0
Accuracy : 0.9958
Specificity : 0.9968
AUC-ROC : 0.4984
AUC-PR : 0.0005

Metrics for Logistic Regression:
Precision : 0.0074
Recall : 0.6216
F1 Score : 0.0147
Accuracy : 0.9173
Specificity : 0.9176
AUC-ROC : 0.7696
AUC-PR : 0.3147

Metrics for K-Nearest Neighbors:
Precision : 0.0233
Recall : 0.0541
F1 Score : 0.0325
Accuracy : 0.9968
Specificity : 0.9977
AUC-ROC : 0.5259
AUC-PR : 0.0391

Metrics for Neural Network:
Precision : 0.0143
Recall : 0.0541
F1 Score : 0.0226
Accuracy : 0.9954
Specificity : 0.9963
AUC-ROC : 0.5252
AUC-PR : 0.0346

[CV] END .................C=0.1, gamma=scale, kernel=sigmoid; total time=15.7min
[CV]

## Takeaways from each Metric

* Precision (TPR): This can be thought of as the number of failed devices that were correctly predicted as faulted by our model, in relation to the to total number of devices were predicted to be faulted by our model. This was highest by far for gradient boosting, with a precision of 16.67%. So when looking at the total pool of samples predicted to be a "1", or faulted, about 16.67% of them were correct for the best model.

* Recall. This can be seen as the number of devices that were correctly predicted to be faulted, in relation to the total number of devices that are actually faulted. This was highest by far with our logisitc regression model. So when looking at the total pool of faulted devices, in the best case, we identified 62% of them.

* AUC-ROC: This is a measurement that takes into account both TPR and FPR. AUC-ROC can be biased towards the majority class when there is a class imbalance because it emphasizes false positives and true negatives. 

* AUC-PR: AUC-PR, on the other hand, focuses on the trade-off between precision and recall. AUC-PR is generally more informative when dealing with imbalanced datasets because it evaluates the classifier's performance based on positive class identification, which is often more critical in such scenarios

* The Logisitic Regression Model dominated both of these metrics, of .77 and .31. Overall it had a much lower accuracy relative to the other models, which seems to indicate that it was more likely to a positive label (faulted) for samples.